In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
import torch.utils.data
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets, models
import os
import time
import copy

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/kaggle/input/hymenoptera-data/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def train_model_N(model, criterion, optimizer, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:       
            if phase == 'train':
#                 scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':       #  不是训练的话就不用计算梯度
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:        # 保存最好的模型参数数据
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
#     model.load_state_dict(best_model_wts)
    return model

# 训练一： 冻结模型参数，运用L2正则

In [5]:
model_18 = models.resnet18(pretrained=True) 

for p,name in zip(model_18.parameters(),model_18.state_dict().keys()):
    p.requires_grad=False
    
model_18.fc = nn.Linear(model_18.fc.in_features,2)

for p in model_18.fc.parameters():
    p.requires_grad = True
    
# for p,name in zip(model_18.parameters(),model_18.state_dict().keys()):
#     print(name,':',p.requires_grad)
    
model_18 = model_18.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model_18.parameters(),weight_decay=0.001)

model_18_train = train_model_N(model_18,criterion,optimizer_ft, num_epochs=10)

Epoch 0/9
----------
train Loss: 0.5739 Acc: 0.7213
val Loss: 0.2853 Acc: 0.9346

Epoch 1/9
----------
train Loss: 0.4401 Acc: 0.8033
val Loss: 0.2888 Acc: 0.8758

Epoch 2/9
----------
train Loss: 0.4109 Acc: 0.8197
val Loss: 0.1984 Acc: 0.9412

Epoch 3/9
----------
train Loss: 0.5035 Acc: 0.7459
val Loss: 0.2038 Acc: 0.9346

Epoch 4/9
----------
train Loss: 0.4263 Acc: 0.7951
val Loss: 0.1799 Acc: 0.9477

Epoch 5/9
----------
train Loss: 0.4295 Acc: 0.8197
val Loss: 0.1803 Acc: 0.9346

Epoch 6/9
----------
train Loss: 0.3826 Acc: 0.8279
val Loss: 0.2191 Acc: 0.9346

Epoch 7/9
----------
train Loss: 0.3991 Acc: 0.8033
val Loss: 0.1618 Acc: 0.9477

Epoch 8/9
----------
train Loss: 0.5066 Acc: 0.7541
val Loss: 0.1782 Acc: 0.9346

Epoch 9/9
----------
train Loss: 0.4311 Acc: 0.8279
val Loss: 0.1601 Acc: 0.9477

Training complete in 1m 5s
Best val Acc: 0.947712


# 训练二： 用模型参数初始化训练，调用L2正则

In [6]:
model_18 = models.resnet18(pretrained=True) 

model_18.fc = nn.Linear(model_18.fc.in_features, 2)

model_18 = model_18.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model_18.parameters(),weight_decay=0.001)

model_18_train = train_model_N(model_18,criterion,optimizer_ft, num_epochs=10)

Epoch 0/9
----------
train Loss: 1.2326 Acc: 0.5574
val Loss: 0.8500 Acc: 0.4118

Epoch 1/9
----------
train Loss: 0.7875 Acc: 0.4918
val Loss: 0.6612 Acc: 0.4575

Epoch 2/9
----------
train Loss: 0.7142 Acc: 0.5820
val Loss: 3.7550 Acc: 0.5621

Epoch 3/9
----------
train Loss: 0.7266 Acc: 0.4713
val Loss: 0.7786 Acc: 0.4444

Epoch 4/9
----------
train Loss: 0.7016 Acc: 0.5574
val Loss: 0.8894 Acc: 0.5294

Epoch 5/9
----------
train Loss: 0.7062 Acc: 0.5615
val Loss: 0.7371 Acc: 0.5621

Epoch 6/9
----------
train Loss: 0.7250 Acc: 0.5820
val Loss: 0.7292 Acc: 0.5817

Epoch 7/9
----------
train Loss: 0.7170 Acc: 0.5656
val Loss: 0.6568 Acc: 0.6405

Epoch 8/9
----------
train Loss: 0.6955 Acc: 0.5984
val Loss: 1.1515 Acc: 0.5359

Epoch 9/9
----------
train Loss: 0.7114 Acc: 0.5820
val Loss: 0.7658 Acc: 0.4837

Training complete in 1m 7s
Best val Acc: 0.640523


# 冻结模型参数，L1正则化训练全连接层

In [7]:

def train_model_L1(model,criterion,optimizer,num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):        # epochs循环
        print("Epoch{}/{}".format(epoch,num_epochs-1))
        print("-"*10)
        
        for phase in ["train","val"]:     # 训练模型还是检测模型
            if phase == "train":
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:    # 数据装载位置
                inputs = inputs.to(device)
                labels = labels.to(device)   # 把数据装载过去

                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs,1)
                    
                    L1 = 0
                    for param in model.parameters():
                        L1 += torch.sum(torch.abs(param))
                        
                    loss = criterion(outputs,labels) + 0.001 * L1
                    
                    
                    if phase == "train":

                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss:{:.4f} Acc:{:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
    time_elapsed = time.time()-since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.4f}'.format(best_acc))

    # load best model weights
#     model.load_state_dict(best_model_wts)
    return model

In [8]:
model_18 = models.resnet18(pretrained=True) 



for p,name in zip(model_18.parameters(),model_18.state_dict().keys()):
    p.requires_grad=False
    
model_18.fc = nn.Linear(model_18.fc.in_features,2)

for p in model_18.fc.parameters():
    p.requires_grad = True
    
# for p,name in zip(model_18.parameters(),model_18.state_dict().keys()):
#     print(name,':',p.requires_grad)

model_18 = model_18.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model_18.parameters())

model_18_train = train_model_L1(model_18,criterion,optimizer_ft, num_epochs=10)

Epoch0/9
----------
train Loss:168.4047 Acc:0.7008
val Loss:168.1198 Acc:0.8954
Epoch1/9
----------
train Loss:168.3342 Acc:0.7418
val Loss:168.0437 Acc:0.9281
Epoch2/9
----------
train Loss:168.2733 Acc:0.7787
val Loss:168.0789 Acc:0.8954
Epoch3/9
----------
train Loss:168.3101 Acc:0.7295
val Loss:168.0347 Acc:0.9020
Epoch4/9
----------
train Loss:168.2056 Acc:0.7951
val Loss:168.0044 Acc:0.9346
Epoch5/9
----------
train Loss:168.1903 Acc:0.8156
val Loss:167.9893 Acc:0.9477
Epoch6/9
----------
train Loss:168.2224 Acc:0.7951
val Loss:168.0561 Acc:0.9020
Epoch7/9
----------
train Loss:168.1500 Acc:0.8320
val Loss:168.0955 Acc:0.8824
Epoch8/9
----------
train Loss:168.2398 Acc:0.7869
val Loss:168.0147 Acc:0.9412
Epoch9/9
----------
train Loss:168.2346 Acc:0.7992
val Loss:167.9974 Acc:0.9542
Training complete in 1m 4s
Best val Acc: 0.9542


## 结论：
1. 由于图像具有一定的规律，运用模型自带的参数已经解析了图像自我的规律，所以运用冻结后的参数训练，有利于提高模型的准确率

2. 在本次实验中，L1正则的效果优于L2正则，主要原因是在已经冻结了模型的参数的基础上，L1的稀疏性能更好的在最后的全连接层中正确的分类。